# Sample of using image pull secrets
ImagePullSecrets is a way provided by kubernetes to access hidden/protected images. It is a list of references to secrets. Please refer to [here](https://kubernetes.io/docs/concepts/containers/images/#specifying-imagepullsecrets-on-a-pod) and [Argo reference](https://github.com/argoproj/argo/blob/47eba519107c229edf61dbe024a6a5e0f1618a8d/pkg/apis/workflow/v1alpha1/types.go#L115) for more details.

In KFP, we can simply set the ImagePullSecrets for each pipeline to grant it the ability to access the images in other registries when needed. Please see the example below for a simple demonstration.

In [ ]:
!pip3 install kfp --upgrade

In [6]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kubernetes import client as k8s_client

In [7]:
class GetFrequentWordOp(dsl.ContainerOp):
  """A get frequent word class representing a component in ML Pipelines.
  """
  def __init__(self, name, message):
    """Args:
         name: An identifier of the step which needs to be unique within a pipeline.
         message: a dsl.PipelineParam object representing an input message.
    """
    super(GetFrequentWordOp, self).__init__(
        name=name,
        image='python:3.5-jessie',
        command=['sh', '-c'],
        arguments=['python -c "from collections import Counter; '
                   'words = Counter(\'%s\'.split()); print(max(words, key=words.get))" '
                   '| tee /tmp/message.txt' % message],
        file_outputs={'word': '/tmp/message.txt'})

In this example, we defined a KFP component by directly specifying the base image and the python command it runs. Note that we're using a public image so actually there is no need to use imagepullsecrets. However, if the image lives within a private/protected container registry, an imagepullsecret might be necessary to authenticate the pipeline in order to pull the image.

In [8]:
@dsl.pipeline(
  name='Save Most Frequent',
  description='Get Most Frequent Word and Save to GCS'
)
def save_most_frequent_word(message: str):
  """A pipeline function describing the orchestration of the workflow."""

  counter = GetFrequentWordOp(
          name='get-Frequent',
          message=message)
  dsl.get_pipeline_conf().set_image_pull_secrets([k8s_client.V1ObjectReference(name="secretA")])

In [10]:
# Compile the pipeline.
compiler.Compiler().compile(save_most_frequent_word,  'save_most_frequent.yaml')

The above code should generate following `save_most_frequent.yaml` pipeline definition.
```yaml
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: save-most-frequent-
spec:
  arguments:
    parameters:
    - name: message
  entrypoint: save-most-frequent
  imagePullSecrets:
  - name: secretA
  serviceAccountName: pipeline-runner
  templates:
  - container:
      args:
      - python -c "from collections import Counter; words = Counter('{{inputs.parameters.message}}'.split());
        print(max(words, key=words.get))" | tee /tmp/message.txt
      command:
      - sh
      - -c
      image: python:3.5-jessie
    inputs:
      parameters:
      - name: message
    name: get-frequent
    outputs:
      artifacts:
      - name: mlpipeline-ui-metadata
        optional: true
        path: /mlpipeline-ui-metadata.json
      - name: mlpipeline-metrics
        optional: true
        path: /mlpipeline-metrics.json
      parameters:
      - name: get-frequent-word
        valueFrom:
          path: /tmp/message.txt
  - dag:
      tasks:
      - arguments:
          parameters:
          - name: message
            value: '{{inputs.parameters.message}}'
        name: get-frequent
        template: get-frequent
    inputs:
      parameters:
      - name: message
    name: save-most-frequent
```

When looking into the `save_most_frequent.yaml` generated, one can find that the generated workflow is associated with an imagePullSecrets specified via its name.